This is a short notebook to walk through some of the applications of Transformer neural nets for tokenized analytic data, and demonstrate the functionality of the repo through examples. First let's load in the necessary libraries and modules.

In [ ]:
import encoder_decoder, encoder_only, decoder_only

Each architecture does something a little different, roughly, the inputs and outputs look like the following:

1. Encoder-Only: [3,4,2,5,...,5,6,3,4,3,3] --> 8
2. Decoder-Only: [5,6,3,4,3] --> [5,6,3,4,3,3]
3. Encoder-Decoder: [5,6,3,4,3,3] --> [3,4,2,5,1,3,2]

In words, this looks like:

1. Encoders take a sequence and maps it to a new vector in the embedding space that gets mapped to a single category
2. Decoders take a sequence and predict the next token, either unconditionally, or...
3. Encoder-Decoder conditions the next token predictino of the decoder layers with an encoder output vector. 

Obviously, these are all overlappping, and in many ways you can create the same behavior for encoders with decoders and vice-versa (just have the output of encoder map to the next token in the sequence, as opposed to some completely different semantic category). 

But for historical reasons, we'll keep all three of these architectures distinct as they have been used for different types of token prediction tasks.

Let's start with encoder only and "train" a neural network to identify the largest token in a sequence -- i.e. effectively implement a MAX function acting on list using a neural network.

In [29]:
encoder_decoder.importLibs()
import torch
from torch.utils.data import DataLoader

Finished import


In [13]:
# Define Dataset Class
class SequenceDataset(torch.utils.data.Dataset):
    def __init__(self, sequences, targets):
        self.sequences = sequences
        self.targets = targets
    
    def __len__(self):
        return len(self.sequences)
    
    def __getitem__(self, idx):
        return torch.tensor(self.sequences[idx], dtype=torch.long), torch.tensor(self.targets[idx], dtype=torch.long)

In [ ]:
seq = []
tgt = []
import random

for i in range(10):
    start = []
    for j in range(5):
        start.append(random.randint(1,10))
    seq.append(start)
    tgt.append(random.randint(1,20))

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
data = SequenceDataset([[3,4,5]],[3])

[[8, 7, 2, 8, 3], [5, 7, 5, 1, 4], [4, 2, 10, 6, 2], [7, 2, 3, 9, 2], [2, 2, 9, 6, 2], [8, 1, 7, 2, 9], [5, 5, 8, 4, 7], [6, 5, 2, 9, 2], [9, 7, 3, 4, 3], [6, 3, 6, 10, 5]]


In [48]:
loader=DataLoader(data,batch_size=1, shuffle=True)
for epoch in range(5):
    for seq,tgt in loader:
        seq,tgt = seq.to(device), tgt.to(device)
        print(seq[0],tgt[0])

tensor([3, 4, 5]) tensor(3)
tensor([3, 4, 5]) tensor(3)
tensor([3, 4, 5]) tensor(3)
tensor([3, 4, 5]) tensor(3)
tensor([3, 4, 5]) tensor(3)
